In [1]:
import pandas as pd
import numpy as np

import swifter
import os
from cleanco import cleanco
import re
import json
import pycountry

In [29]:
#Text preprocessing
def name_preprocess(df, name):
    df[name] = df[name].astype(str)

    # Convert to lowercase
    df[name] = df[name].swifter.apply(lambda x: x.lower())

    # Remove commas
    df[name] = df[name].swifter.apply(lambda x: x.replace(",", ""))
    
    # Remove dot
    df[name] = df[name].swifter.apply(lambda x: x.replace(".", ""))
    
    # Remove *
    df[name] = df[name].swifter.apply(lambda x: x.replace("*", ""))
    
    # Remove !
    df[name] = df[name].swifter.apply(lambda x: x.replace("!", ""))
    
    # Remove ''
    df[name] = df[name].swifter.apply(lambda x: x.replace("'", ""))
    
    # Remove ;
    df[name] = df[name].swifter.apply(lambda x: x.replace(";", ""))
    
    # Remove /
    df[name] = df[name].swifter.apply(lambda x: x.replace("/", ""))

    # Remove hyphens
    df[name] = df[name].swifter.apply(lambda x: x.replace("-", ""))
    
    df[name] = df[name].swifter.apply(lambda x: x.replace("(", ""))
    
    df[name] = df[name].swifter.apply(lambda x: x.replace(")", ""))
    
    df[name] = df[name].swifter.apply(lambda x: x.replace("&", ""))

    # Remove text between parenthesis
    df[name] = df[name].swifter.apply(lambda x: x.replace(r"\(.*\)", ""))

    # Remove spaces in the begining/end
    df[name] = df[name].swifter.apply(lambda x: x.strip())
    

    return df

In [3]:
#standardized company names in the crosswalk
#standardize words
replacements = {'ac': 'action', 'assn': 'association', 'ass' : 'association', 'assoc': 'association',  'associations': 'association', 'associates': 'associate',
    'cmte': 'committee',  'comm':'committee', 'com': 'committee',  'corp':'corporation','coporations': 'corporation', 'co':'company',
    'companies':'company', 'citizen': 'citizens',  'employee': 'employees',  'enterprises': 'enterprise', 'fir': 'firm', 'grou':'group', 
     'govt': 'government', 'inst': 'institute', 'leag':'league', 'org': 'organization', 'organizations': 'organization',}

def replace(match):
    return replacements[match.group(0)]

def replace_words(s):
    return re.sub('|'.join(r'\b%s\b' % re.escape(s) for s in replacements),  replace, s) 

In [4]:
end_words_company_name = ['association', 'bancorp', 'bancorporation', 'bank', 'company', 'corporate', 'corporation', 
                          'council','enterprise','foundation','group','inc','incorporated','institute',  'll, ''llc', 'llp', 'ltd', 'management', 'organization', 
                          'partners', 'pllc', 'services', 'utitlies']

#cut words after word in the end_word_company_name list 
def cut_end_words(s, end_words_company_name_ = end_words_company_name):
    for name in end_words_company_name_:
        s = s.split(' ' + name + ' ')[0]
    return s

In [5]:
#remove stop words
#remove words
    
drop_words_list  = ['alabama', 'alabaman', 'alaska', 'alaskan', 'america', 'american', 'arizona', 'arizonan', 'arkansas', 'arkansasn', 'attorneys',
                    'associate', 'association','bancorp', 'bancorporation', 'california', 'californian', 'center', 'ctr', 'company', 'coloradan', 'colorado',  'company', 'connecticut', 'connecticutan', 'corporation', 'delaware',
 'construction', 'delawarean', 'employees','enterprise', 'family',  'families', 'federal', 'firm', 'florida', 'floridan', 'georgia', 'georgian',
                    'group', 'hawaii', 'hawaiian', 
                    'holdings', 'idaho', 'idahoan', 'illinois', 'illinoisan',
 'inc', 'incorporated', 'institute', 'indiana', 'indianan', 'international','iowa','iowan','kansas','kansasan','kentucky','kentuckyan', 'll', 'llc', 'llp',
 'louisiana', 'louisianan', 'ltd', 'maine',  'mainean', 'maryland', 'marylandan', 'massachusetts', 'massachusettsan', 'michigan',
 'michiganan', 'mid us', 'minnesota','minnesotan','mississippi','mississippian', 'missouri', 'missourian', 'montana','montanan',
 'national', 'nebraska', 'nebraskan', 'nevada', 'nevadan','new hampshire','new jersey','new mexico','new york',
 'nonpartisan', 'non partisan', 'north america', 'north carolina', 'north carolinians', 'north dakota', 'north dakotan', 'office',
 'ohio', 'ohioan', 'oklahoma', 'oklahoman', 'operations', 'operating', 'oregon', 'oregonan', 'pac', 'partnership', 'pennsylvania','pennsylvanian','pllc',
                    'properties', 'rhode island',
 'rhode islandan','industries', 'service', 'services', 'south carolina', 'south carolinian', 'south dakota', 'south dakotan', 'supply', 'tennesean',
 'tennessee','texan','texas', 'united states', 'us', 'usa', 'utah', 'utahan','vermont','vermontan','virginia','virginian','washington',
 'washingtonan','west virginia', 'west virginian', 'wisconsin','wisconsinan', 'wyoming', 'wyomingan', 'svc', 'medical', 'solutions', 'energy', 'air', 'conditioning', 'research' , 'technology', 'los' , 'angeles',
                'new york', 'guangzhou', 'beijing', 'shanghai', 'chicago','fuel', 'power', 'medical', 'products', 
                'systems', 'rotterdam', 'university', 'state', 'agricultural', 'universidad', 'generation', 'foods',
                'food','semiconductor','acquisition', 'instrument', 'university', 'college', 'investment',
                'east', 'west', 'europe', 'data','health', 'data', 'animal', 'performance', 'design', 'innovations', 'hk']

stopwords = ['the', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very',
            'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other',
             'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these',
             'your', 'his', 'through', 'don', 'nor',  'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up',
             'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can',
             'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those',
         'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

drop_words_list1 = drop_words_list.copy()
drop_words_list1.extend(stopwords)

for i in list(pycountry.countries):
    drop_words_list1.append(i.alpha_2.lower())
    drop_words_list1.append(i.alpha_3.lower())
    drop_words_list1.append(i.name.lower())
    


def remove_word(x):
    x = x.split()
    resultwords  = [word for word in x if word not in drop_words_list1]
    result = ' '.join(resultwords)
    return result


In [6]:
def process_company_dataset(df_company2014, columnname = 'company'):
    df_company2014 = name_preprocess(df_company2014, columnname)
    df_company2014[columnname] = df_company2014[columnname].swifter.apply(replace_words)
    df_company2014[columnname] = df_company2014[columnname].swifter.apply(cut_end_words)
    df_company2014[columnname] = df_company2014[columnname].swifter.apply(remove_word)



    #no need to drop duplicated values

    df_company2014[columnname]  = df_company2014[columnname].swifter.apply(lambda x: x.strip())

    df_company2014[columnname] = df_company2014[columnname].swifter.apply(lambda x: x.strip())

    df_company2014[columnname]  = df_company2014[columnname].swifter.apply(lambda x: x.strip())
    
    df_company2014[columnname]  = df_company2014[columnname].swifter.apply(lambda x: x.strip())

    df_company2014[columnname] = df_company2014[columnname].swifter.apply(lambda x: x.strip())
    
    #drop empty company values    
    return df_company2014



In [7]:
def process_company_dataset_1(df_company2014, columnname = 'company'):
    df_company2014[columnname] = df_company2014[columnname].swifter.apply(lambda x: cleanco(x).clean_name())
    df_company2014[columnname] = df_company2014[columnname].swifter.apply(lambda x: cleanco(x).clean_name())
    #drop empty company values    
    return df_company2014

In [ ]:
#process patent transfer

In [ ]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\notmeaningfultransfer")

In [ ]:
files = os.listdir()
files

In [ ]:
file = pd.read_csv('2013_01_notmeaningfultransfer.csv')
file['assignor'] = file['assignor'].astype(str)
companynames = np.unique(file['assignor'].dropna().values)

for i in file.columns:
    if i[0:4] == 'name':
        file[i] = file[i].astype(str)
        companynames = np.concatenate((companynames, np.unique(file[i].dropna().values)))
companynames = np.unique(companynames)

In [ ]:
for file in files[1:]:
    file = pd.read_csv(file)
    file['assignor'] = file['assignor'].astype(str)
    companynames = np.concatenate((companynames, np.unique(file['assignor'].dropna().values)))
    for i in file.columns:
        if i[0:4] == 'name':
            file[i] = file[i].astype(str)
            companynames = np.concatenate((companynames, np.unique(file[i].dropna().values)))
    companynames = np.unique(companynames)

In [2]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\identifycompanypermno\\intermediate_output")

In [ ]:
companynames = pd.DataFrame(companynames, columns = ['company'])
companynames.to_csv("patenttransfer_companynames.csv", index = False)

In [ ]:
companynames['rawcompany'] = companynames['company']

In [ ]:
companynames = process_company_dataset(companynames, 'company') 
companynames= process_company_dataset_1(companynames, 'company') 

In [ ]:
companynames.loc[companynames[companynames['company'] == ''].index, 'company'] = companynames[companynames['company'] == '']['rawcompany']
companynames.loc[companynames[companynames['company'].isna()].index, 'company'] = companynames[companynames['company'].isna()]['rawcompany']

In [ ]:
companynames.to_csv("patenttransfer_companynames_standardized.csv", index = False)

In [10]:
#process first patent assignment
firstpatentassignments  = pd.read_csv("first_patent_assignment_1980_2015.csv")

In [11]:
firstpatentassignments['name_0'] = firstpatentassignments['name_0'].astype(str)
firstpatentassignments['rawname_0'] = firstpatentassignments['name_0']

In [14]:
name_0_unique = np.unique(firstpatentassignments['name_0'].values)

In [22]:
name_0_unique = pd.DataFrame(name_0_unique, columns = ['name_0'])

In [30]:
name_0_unique = process_company_dataset(name_0_unique, 'name_0') 
name_0_unique= process_company_dataset_1(name_0_unique, 'name_0') 

In [31]:
name_0_unique_raw_dict = dict(zip(list(name_0_unique['rawname_0'].values),list(name_0_unique['name_0'].values)))

In [32]:
firstpatentassignments['name_0']  = firstpatentassignments['rawname_0'].map(name_0_unique_raw_dict)

In [45]:
firstpatentassignments.loc[firstpatentassignments[firstpatentassignments['name_0']  == ''].index, 'name_0'] = firstpatentassignments[firstpatentassignments['name_0']  == '']['rawname_0']
firstpatentassignments.loc[firstpatentassignments[firstpatentassignments['name_0'].isna()].index, 'name_0'] = firstpatentassignments[firstpatentassignments['name_0'].isna()]['rawname_0']

In [50]:
firstpatentassignments = firstpatentassignments[~firstpatentassignments['rawname_0'].isna()]

In [51]:
firstpatentassignments.to_csv("first_patent_assignment_1980_2015_standardized.csv", index= False)

In [4]:
assignments = pd.read_csv("first_patent_assignment_1980_2015_standardized.csv")